In [1]:
# -*- coding: utf-8 -*-

"""
note something here

"""

__author__ = 'Wang Chen'
__time__ = '2019/7/15'

import copy
from queue import PriorityQueue

import pandas as pd
import numpy as np

from Item import Item
from Event import Event
from Hypercube import Hypercube
#from static import *


In [2]:
class Popcache:
    """
    dim: 维度
    item_list: 视频列表，主键为视频的id
    hypercube_array: 立方空间的dim维数组
    max_day: 最大天数
    cur_day: 当天天数
    all_UITs: np.adarray 从pandas转化来的原始值
    event_lastday_list: 前一天的事件缓存列表
    event_count: 事件计数
    cache_size: 缓存列表大小
    cache_list: 缓存列表，存储视频id
    cache_set:

    if_disp: 是否展示
    """
    def __init__(self, dim, max_item_id, max_day, cur_day, all_UITs, cache_size, if_disp):
        """
        dim: 维度
        max_item_id: 最大视频id
        max_day: 最大天数
        cur_day: 当天天数
        all_UITs: np.adarray 从pandas转化来的原始值
        event_lastday_list: 前一天的事件缓存列表
        cache_size: 缓存列表大小

        if_disp: 是否展示
        """
        self.dim = dim
        self.item_list = [Item(item_id) for item_id in range(max_item_id)]
        self.hypercube_array = self.init_hypercube_array(dim)
        self.max_day = max_day
        self.cur_day = cur_day
        self.all_UITs = all_UITs
        self.curday_values = None
        self.event_lastday_list = []
        self.event_count = -1
        self.cache_size = cache_size
        self.cache_list = PriorityQueue()
        self.cache_set = set()

        self.if_disp = if_disp

    def init_hypercube_array(self, dim):
        """
        构建hypercube空间
        :param dim:
        :return:
        """
        hypercube_array = [Hypercube() for _ in range(2)]
        for _ in range(dim - 1):
            hypercube_array = [copy.deepcopy(hypercube_array) for _ in range(2)]
        return hypercube_array

    def enumerate_hypercubes(self):
        hypercubes_list = self.hypercube_array
        for depth in range(self.dim-1):
            tmp_list = []
            for element in hypercubes_list:
                tmp_list.extend(element)
            hypercubes_list = tmp_list

        if self.if_disp:
            print("enumerate {} depth, total length {}".format(dim, hypercubes_list.__len__()))

        return hypercubes_list

    def get_one_day_values(self):
        """
        获取一天的ndarray
        :return:
        """
        filter_UITs = self.all_UITs[self.all_UITs[:, 3] == self.cur_day]

        if self.if_disp:
            print("cur day {}".format(self.cur_day), end=" , ")
            print("total events {}".format(filter_UITs.shape[0]))
        return filter_UITs

    def extract_feature(self, item_id):
        """
        通过item_id获取视频的特征
        :param item_id:
        :return:
        """
        if self.cur_day > self.max_day:
            raise Exception("days exceed")

        cur_item = self.item_list[item_id]
        if np.sum(cur_item.history) == 0:
            return np.zeros((self.dim,), dtype=np.int32)

        feature = [0] * self.dim
        for pos, feature_day in enumerate(feature_day_list):
            if self.cur_day >= feature_day:
                check = np.sum(cur_item.history[self.cur_day - feature_day:self.cur_day])
            else:
                check = np.sum(cur_item.history[0:self.cur_day])
            if check > 0:
                feature[pos] = 1

        if self.if_disp:
            print("get the feature: ", np.array(feature, dtype=np.int32))

        return np.array(feature, dtype=np.int32)

    def update_feature(self):
        """
        根据ndarray更新当天视频的特征值
        :return:
        """
        count = 0
        for value_line in self.curday_values:
            item_id = value_line[2]
            self.item_list[item_id].history[self.cur_day] += 1
            count += 1

        item_count = 0
        for item in self.item_list:
            if item.history[self.cur_day]:
                item.popularity = item.history[self.cur_day]
                item_count += 1

        if self.if_disp:
            print("total deal with {} events, update {} items".format(count, item_count))

        return item_count

    def update_hypercube_estimate_value(self):
        """
        根据event_list列表更新hypercube他们的MN值
        :return:
        """

        total_popularity = 0

        if self.if_disp:
            print("update hypercube M N")

        for event in self.event_lastday_list:
            video_id = event.item
            event_feature = self.extract_feature(video_id)
            cube = self.select_hypercube(event_feature)
            popularity_plus = cube.update_popularity(self.item_list[video_id].popularity)
            total_popularity += popularity_plus

            if self.if_disp:
                print("current hypercube ", event_feature, end=" , ")
                print("popularity plus {}".format(popularity_plus))

        return total_popularity

    def update_cache_set(self):
        tmp_list = PriorityQueue()
        while not self.cache_list.empty():
            (old_priority, old_item) = self.cache_list.get()
            event_feature = self.extract_feature(old_item)
            cube = self.select_hypercube(event_feature)
            update_estimate_popularity = cube.get_popularity()
            tmp_list.put((update_estimate_popularity, old_item))

        if tmp_list.qsize() != self.cache_set.__len__():
            raise Exception("update heap error")

        self.cache_list = tmp_list

        for event in self.event_lastday_list:
            if event.item in self.cache_set:
                continue

            if len(self.cache_set) < self.cache_size:
                self.cache_set.add(event.item)
                self.cache_list.put((event.esti_popularity, event.item))
            else:
                (top_priority, top_item) = self.cache_list.queue[0]

                if event.esti_popularity >= top_priority:  # 替换条件
                    (replace_popularity, replace_item) = self.cache_list.get()
                    self.cache_set.remove(replace_item)
                    self.cache_list.put((event.esti_popularity, event.item))
                    self.cache_set.add(event.item)
                    # print("{} out and {} in".format(replace_popularity, event.esti_popularity))
        return True

    def update_one_day(self):
        """
        天数加1
        :return:
        """
        self.cur_day += 1
        self.event_lastday_list = []
        if self.cur_day > self.max_day:
            raise Exception("days exceed")

        if self.if_disp:
            print("one day adances, current day {}".format(self.cur_day))

        return self.cur_day

    def select_hypercube(self, feature):
        """
        根据视频的特征选择对应的hypercube
        :param feature:
        :return:
        """
        hypercube = self.hypercube_array
        for i in feature:
            hypercube = hypercube[i]

        assert isinstance(hypercube, Hypercube)

        if self.if_disp:
            print("current hypercube: ", feature)

        return hypercube

    def estimate_popularity(self, hypercube, event):
        """
        根据hypercube的总体估计当前事件的流行度
        :param hypercube:
        :param event_id:
        :return:
        """
        hypercube.add_event(event)       # 添加一个事件
        popularity = hypercube.get_popularity()     # 获取popularity
        event.esti_popularity = popularity

        if self.if_disp:
            print("the esti_popularity is {}".format(popularity))

        return popularity

    def event_add_oneday(self,curValues):
        """
        添加cur_day当天的event进event_list
        """
        #self.curday_values = self.get_one_day_values()
        self.curday_values = curValues
        for value_line in self.curday_values:
            self.event_count += 1
            event = Event(id=value_line[0], user=value_line[1], item=value_line[2], occur_time=value_line[3], esti_popularity=0)
            self.event_lastday_list.append(event)

        if self.if_disp:
            print("today add {} events, current event pos: {}".format(self.curday_values.shape[0], self.event_count))    # for check use

        return self.event_lastday_list

    def curday_event_into_cube(self, is_validate=False):
        count = 0
        hit = 0
        for event in self.event_lastday_list:

            if self.if_disp:
                print("estimate event {}".format(event.id))

            event_feature = self.extract_feature(event.item)
            cube = self.select_hypercube(event_feature)
            _estimate_popularity = self.estimate_popularity(cube, event)

            hit += self.metric(event)
            count += 1

        if self.if_disp:
            print("total estimate {} events".format(count))

        if is_validate:
            print("today hit rate is {}".format(round(hit/count, 3)))

        return count, hit

    def print_cubes(self):
        print("day {} show cubes".format(self.cur_day))
        hypercubes_list = self.enumerate_hypercubes()

        for pos, hypercube in enumerate(hypercubes_list):
            print("current cube {}".format(pos), end=" , ")
            print("M:{} N:{}".format(hypercube.M, hypercube.N), end=" , ")
            print(hypercube.get_popularity())

    def metric(self, event):
        """
        最小堆比较，缓存击中返回1，否则返回0；
        :param event:
        :return:
        """
        hit = 0
        if event.item not in self.cache_set:
            # if len(self.cache_set) < self.cache_size:
            #     self.cache_set.add(event.item)
            #     self.cache_list.put((event.esti_popularity, event.item))
            # else:
            #     (top_priority, top_item) = self.cache_list.queue[0]
            #
            #     if event.esti_popularity >= top_priority:  # 替换条件
            #         (replace_popularity, replace_item) = self.cache_list.get()
            #         self.cache_set.remove(replace_item)
            #         self.cache_list.put((event.esti_popularity, event.item))
            #         self.cache_set.add(event.item)
                    # print("{} out and {} in".format(replace_popularity, event.esti_popularity))
            return hit
        else:
            hit = 1
            return 1

In [3]:
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
import time

import collections
import copy

In [47]:
class ENV(object):
    def __init__(self,userNum,contentNum,latency,Bu):
        self.userNum = userNum
        self.contentNum =contentNum

        self.r = np.zeros(shape=(userNum,contentNum),dtype=int)
        self.p = np.full(shape=contentNum,fill_value = 1/userNum,dtype=np.float32)
        self.e = np.zeros(shape=contentNum,dtype=int)
        self.S = np.ones(shape=contentNum,dtype=int)
        self.l = np.array(latency,dtype=np.float32)
        

        self.B = np.full(shape=userNum,fill_value=Bu,dtype=int)

        self.pipe = collections.OrderedDict()


    #有序字典实现LRU
    def updateEgdeCache(self,action,t):
        for i in np.argwhere(action==1).squeeze(-1):
            if i in self.pipe.keys():
                self.pipe.pop(i)
            elif len(self.pipe) >= 500:
                self.e[self.pipe.popitem(last=False)[0]] = 0
            self.pipe[i] = t
            self.e[i] = 1

    
    def updateEnv(self,u,action,t):
        
        p_tmp = ((self.r[u] | action)-self.r[u])*(1/self.userNum) + self.p
        self.p = np.where(p_tmp<1-1/self.userNum,p_tmp,1-1/self.userNum)
        self.r[u] =  self.r[u] | action

        self.updateEgdeCache(action,t)

    def getStatus(self):
        return (self.r,
                self.p, 
                self.e,
                self.S,
                self.l)

    #def reset(self):
    #    self.r = np.zeros(shape=(self.userNum,self.contentNum),dtype=int)
    #    self.p = np.full(shape=self.contentNum,fill_value = 1/self.userNum)
    #    self.e = np.zeros(shape=self.contentNum)
    #    self.S = np.ones(shape=self.contentNum,dtype=int)
    #    self.l_edge = 0.1
    #    self.l_cp = 1
    #    self.B = np.full(shape=self.userNum,fill_value=15,dtype=int)
    #    self.pipe = collections.OrderedDict()

class UE(object):
    def __init__(self,u,env,rewardPara):
        self.u = u

        self.W = []
        self.v = np.zeros(contentNum,dtype=int)

        self.Bu = Bu
        self.contentNum = contentNum
        self.userNum = userNum

        r , p , e, S, l = env.getStatus()

        #self.lastAction = torch.zeros(size=(contentNum,),dtype=int)
        self.lastAction = np.zeros(contentNum,dtype=int)

        self.ALPHAh = rewardPara['alpha']
        self.BETAo =  rewardPara['betao']
        self.BETAl =  rewardPara['betal']             
        self.reward = 0

        self.simU = simUsers[u]
        self.lastStatusFeature = torch.from_numpy(self.statusEmbedding(r,p,e))
        #self.lastp = p.clone()
        #self.lastp = p.copy()
        
        self.lastp = p
       # self.lastr = r.clone()
        #self.lastr = r.copy()
        self.lastru = r[self.u]
        
    def updateViewContent(self,i):
        self.W.append(i)
        self.v[i] = 1

    def statusEmbedding(self,r,p,e):
        tmp = np.zeros(contentNum,dtype=np.float32)
        for simUser in self.simU:
            tmp += r[simUser]
        simUserRu = (tmp / len(self.simU))
        ru = r[self.u]
        statusFeature = np.row_stack((self.v,ru,simUserRu,p,e))
        
        return statusFeature.T.astype(np.float32)
    
    def getReward(self,lastru,lastp,ru,p,i,lastAction,S,l,e,v):
        
        #self.Rh =   self.ALPHAh * (np.log(v * p + (1-v) * (1-p)).sum() / np.log(ru * p + (1-ru) * (1-p)).sum() )

        #self.Rh = self.ALPHAh * ( 1 / ( 1 + np.exp( 0.5 * np.log( ( lastru * lastp + ( 1 - lastru ) * ( 1 - lastp ) ) / ( ru * p + ( 1 - ru ) * ( 1 - p ) ) ).sum() ) ) )

        self.Rh = - self.ALPHAh * ( np.log( ( lastru * lastp + ( 1 - lastru ) * ( 1 - lastp ) ) / ( ru * p + ( 1 - ru ) * ( 1 - p ) ) ).sum() )


        self.Ro =   self.BETAo * lastAction[i] * S[i] * ( e[i] * l[0] + ( 1 - e[i] ) * l[1] )
        
        # self.Ro =   self.BETAo * lastAction[i] * S[i] * ( 1 + e[i] * l[0] + ( 1 - e[i] ) * l[1] )

        self.Rl =   self.BETAl * ( 1 - lastAction[i] ) * S[i] * e[i] * l[2]

        #self.Rh[i] = self.Rh[i] + self.Ro + self.Rl

        #return  self.Rh.sum()
        return  (self.Rh + self.Ro + self.Rl).astype(np.float32)

    def selectAction(self,env,uit,popcache):
        
        self.updateViewContent(uit[1])
        
        r , p , e, S, l = env.getStatus()
        
        self.reward = self.getReward(self.lastru,self.lastp,r[self.u],p,self.W[-1],self.lastAction,S,l,e,self.v)

        self.lastp = p
        self.lastru = copy.copy(r[self.u])
        
        #self.lastp = p.clone()
        #self.lastr = r.clone()
        
        if self.W[-1] not in np.argwhere(self.lastAction): #local cache miss
            cacheSet = popcache.cache_set.copy()
            
            action = np.zeros(contentNum,dtype=int)
            action[self.W[-1]] = 1

            if len(cacheSet)<self.Bu:
                actionIndex = list(torch.randint(0,self.contentNum,(self.Bu,)))
                if self.W[-1] not in actionIndex:
                    actionIndex.pop()
                for index in actionIndex:
                    action[index] = 1
            else:
                if self.W[-1] not in cacheSet:
                    cacheSet.pop() 
                for i in cacheSet:
                    action[i] = 1
            self.lastAction = action
            env.updateEnv(self.u,action,uit[2])
        else:
            action = self.lastAction # keep the cache and no request the new video
        return action
    

In [48]:
level = 'u'
# flag = "不按天更新"
#data_path = '/home/zhangxz/workspace/data/Nati1000_U500_V30/'
#data_path = '/home/ubuntu/data/dataset/R1584_U50_V2/'
data_path = '/home/zhangxz/workspace/dataset/ml-100k/'
#part_one = pd.read_csv(data_path + 'train.csv', header=None)
#part_two = pd.read_csv(data_path + 'test.csv', header=None)
#UIT = pd.concat([part_one, part_two], axis=0)
UIT = pd.read_csv(data_path + 'ml.trace',header= None)
simUsers = np.loadtxt(data_path+"simUser.csv",  delimiter=',',dtype=int)
#import os
#data_path =  '/home/zhangxz/workspace/dataset/ml-100k/'
#if not os.path.exists(data_path):
#    os.makedirs(data_path)
#
#UIT.to_csv(data_path + 'UIT.csv',index=0,header=None)
#UIT = pd.read_csv(data_path + 'UIT.csv',header=None)

#UIT = UIT[UIT['day']<18]
group_count = 0
result_list = []
#UIT.columns = [[0,1,2,3,4,5,6,7,8,9,10,11]]
UIT

,0,1,2,3
0,46,8033,0,2239
1,363,13244,0,3907
2,363,15904,0,4013
3,979,3842,0,5523
4,979,8313,0,5536
...,...,...,...,...
132685,717,8224,179,15549531
132686,717,11752,179,15549743
132687,717,8805,179,15549756
132688,717,5436,179,15549819


In [49]:
contentNum = len(UIT[1].drop_duplicates())
userNum = len(UIT[0].drop_duplicates())

rewardPara = {"alpha":0.01,"betao":1,"betal":1}
latency = [0.2,1,0.8]
Bu = 20


torch.cuda.empty_cache()
UIT.shape

(132690, 4)

In [50]:
UIT_index = UIT.reset_index()
UIT_index

,index,0,1,2,3
0,0,46,8033,0,2239
1,1,363,13244,0,3907
2,2,363,15904,0,4013
3,3,979,3842,0,5523
4,4,979,8313,0,5536
...,...,...,...,...,...
132685,132685,717,8224,179,15549531
132686,132686,717,11752,179,15549743
132687,132687,717,8805,179,15549756
132688,132688,717,5436,179,15549819


In [51]:
all_UITs = UIT.values
items_seq = max(all_UITs[:, 1] + 1)
index = np.arange(0, all_UITs.shape[0]).reshape((all_UITs.shape[0], 1))
all_UITs = np.column_stack((index, all_UITs))
all_UITs

array([[       0,       46,     8033,        0,     2239],
       [       1,      363,    13244,        0,     3907],
       [       2,      363,    15904,        0,     4013],
       ...,
       [  132687,      717,     8805,      179, 15549756],
       [  132688,      717,     5436,      179, 15549819],
       [  132689,      898,    18456,      179, 15551461]])

In [52]:
trainT = 6*18*24
testT = 6*12*24
days = trainT + testT
feature_day_list = [6,6*6,24*6,6*6*24]
print(UIT.shape[0],feature_day_list,days)
dim = 4


132690 [6, 36, 144, 864] 4320


In [53]:
UEs = {}
rewardPara = {"alpha":1,"betao":1,"betal":1}
env = ENV(userNum,contentNum,latency,Bu)
UEs = {}
sumReward = np.zeros(3)
loss = 0
UEHit = np.zeros(userNum)
edgeHit = 0
curDay = 0


cache_size = 20
popcache = Popcache(dim, max_item_id = contentNum, max_day=days, all_UITs=all_UITs,cur_day=curDay,cache_size=cache_size,if_disp=False)

eventIndex = 0
oneDayEvents = []

In [56]:
sumHitrate = np.zeros(UIT.shape[0]// 10000 +2)
UEHitrate = np.zeros(UIT.shape[0]// 10000 +2)
edgeHitrate = np.zeros(UIT.shape[0]// 10000 +2)
privacyReduction = np.zeros(UIT.shape[0]// 10000 +2)

for index,trace in UIT.iterrows():
    
    uit = trace.to_numpy()
    if uit[0] not in UEs:
        UEs[uit[0]] = UE(uit[0],env,rewardPara)
    ue = UEs[uit[0]]
    actionIndex = np.argwhere(ue.lastAction)
    if uit[1] in actionIndex:
        UEHit[uit[0]] += 1
    elif uit[1] in env.pipe.keys():
        edgeHit += 1

    ue.selectAction(env=env,uit=uit,popcache=popcache)
    
    actionIndex = np.argwhere(ue.lastAction)
    for content in actionIndex:
        oneDayEvents.append([eventIndex,uit[0],content,uit[2]])
        eventIndex += 1 
    
    if curDay<uit[2]:
        #print(curDay)
        #print(len(UIT_index[UIT_index[2]==curDay-1].values))
        event_list = popcache.event_add_oneday(np.array(oneDayEvents,dtype=int))
        curDay = uit[2]
        click, hit = popcache.curday_event_into_cube()
        popcache.update_feature()
        total_popularity = popcache.update_hypercube_estimate_value()
        popcache.update_cache_set()
        # popcache.print_cubes()
        # popcache.enumerate_hypercubes()
        popcache.update_one_day()
        popcache.cur_day = curDay
        oneDayEvents = []
        


    sumReward[0] += float(ue.Rh)
    sumReward[1] += float(ue.Rl)
    sumReward[2] += float(ue.Ro)
    if (index+1) % 10000 == 0 :
        psi = 0
        for u in UEs:
            psi += np.log(env.r[u] * env.p + (1-env.r[u]) * (1-env.p)).sum() / np.log(UEs[u].v * env.p + (1-UEs[u].v) * (1-env.p)).sum()
        print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
        print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
        print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
        print()
        sumHitrate[int(index // 10000)]   = round((edgeHit+UEHit.sum())/(index+1),5)
        UEHitrate [int(index // 10000)]   = round(UEHit.sum()/(index+1),5)
        edgeHitrate [int(index // 10000)] = round(edgeHit/(index+1),5)
        privacyReduction [int(index // 10000)] = round(float(psi)/len(UEs),5)
        
psi = 0
for u in UEs:
    psi += np.log(env.r[u] * env.p + (1-env.r[u]) * (1-env.p)).sum() / np.log(UEs[u].v * env.p + (1-UEs[u].v) * (1-env.p)).sum()
print()
print("----------------------------------------------------------------")
print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
print("----------------------------------------------------------------")
print()
sumHitrate [int(round(index / 10000,0))]  = round((edgeHit+UEHit.sum())/(index+1),5)
UEHitrate  [int(round(index / 10000,0))]  = round(UEHit.sum()/(index+1),5)
edgeHitrate[int(round(index / 10000,0))]  = round(edgeHit/(index+1),5)
privacyReduction [int(round(index / 10000,0))] = round(float(psi)/len(UEs),5)

/tmp/ipykernel_9190/3526268193.py:28: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  event_list = popcache.event_add_oneday(np.array(oneDayEvents,dtype=int))


--Time: Sun Oct 17 10:36:41 2021   Index: 9999   Loss: 0.0 --
Reward: [-1.672991e+01  2.214400e-01  1.600000e-04] total reward: -16.50831
UEHitrate: 0.0008  edgeHitrate 0.2768 sumHitrate 0.2776  privacy: 2.36876

--Time: Sun Oct 17 10:38:33 2021   Index: 19999   Loss: 0.0 --
Reward: [-1.080266e+01  2.041600e-01  4.300000e-04] total reward: -10.59807
UEHitrate: 0.00175  edgeHitrate 0.2552 sumHitrate 0.25695  privacy: 1.88524

--Time: Sun Oct 17 10:40:14 2021   Index: 29999   Loss: 0.0 --
Reward: [-8.60326e+00  2.13280e-01  5.00000e-04] total reward: -8.38948
UEHitrate: 0.00183  edgeHitrate 0.2666 sumHitrate 0.26843  privacy: 1.65865

--Time: Sun Oct 17 10:41:45 2021   Index: 39999   Loss: 0.0 --
Reward: [-7.47495e+00  2.05060e-01  4.00000e-04] total reward: -7.26949
UEHitrate: 0.0015  edgeHitrate 0.25633 sumHitrate 0.25783  privacy: 1.52679

--Time: Sun Oct 17 10:43:12 2021   Index: 49999   Loss: 0.0 --
Reward: [-6.71865e+00  2.10220e-01  4.00000e-04] total reward: -6.50803
UEHitrate: 0

In [57]:
sumHitrate, UEHitrate, edgeHitrate

(array([0.2776 , 0.25695, 0.26843, 0.25783, 0.26438, 0.2564 , 0.24971,
        0.24395, 0.24819, 0.24467, 0.25136, 0.25134, 0.24566, 0.24603,
        0.     ]),
 array([0.0008 , 0.00175, 0.00183, 0.0015 , 0.0016 , 0.00152, 0.00141,
        0.00139, 0.00131, 0.0014 , 0.00145, 0.00146, 0.00145, 0.00145,
        0.     ]),
 array([0.2768 , 0.2552 , 0.2666 , 0.25633, 0.26278, 0.25488, 0.2483 ,
        0.24256, 0.24688, 0.24327, 0.24992, 0.24988, 0.24421, 0.24458,
        0.     ]))

In [58]:
privacyReduction

array([2.36876, 1.88524, 1.65865, 1.52679, 1.42193, 1.35381, 1.31732,
       1.27764, 1.2421 , 1.20925, 1.17299, 1.14005, 1.10341, 1.0725 ,
       0.     ])

In [15]:
UEs = {}
rewardPara = {"alpha":1,"betao":0.5,"betal":0.5}
env = ENV(userNum,contentNum)
UEs = {}
sumReward = np.zeros(3)
loss = 0
UEHit = np.zeros(userNum)
edgeHit = 0
curDay = 0

cache_size = 20
popcache = Popcache(dim, max_item_id = contentNum, max_day=days, all_UITs=all_UITs,cur_day=curDay,cache_size=cache_size,if_disp=False)

eventIndex = 0
oneDayEvents = []

In [16]:
sumHitrate = np.zeros(UIT.shape[0]// 10000 +2)
UEHitrate = np.zeros(UIT.shape[0]// 10000 +2)
edgeHitrate = np.zeros(UIT.shape[0]// 10000 +2)
privacyReduction = np.zeros(UIT.shape[0]// 10000 +2)

for index,trace in UIT.iterrows():
    
    uit = trace.to_numpy()
    if uit[0] not in UEs:
        UEs[uit[0]] = UE(uit[0],env,rewardPara)
    ue = UEs[uit[0]]
    actionIndex = np.argwhere(ue.lastAction.numpy())
    if uit[1] in actionIndex:
        UEHit[uit[0]] += 1
    elif uit[1] in env.pipe.keys():
        edgeHit += 1

    ue.selectAction(env=env,uit=uit,popcache=popcache)
    
    actionIndex = np.argwhere(ue.action.numpy())
    for content in actionIndex:
        oneDayEvents.append([eventIndex,uit[0],content,uit[2]])
        eventIndex += 1 

    #print(curDay)
    #print(len(UIT_index[UIT_index[2]==curDay-1].values))
    event_list = popcache.event_add_oneday(np.array(oneDayEvents,dtype=int))
    curDay = uit[2]
    click, hit = popcache.curday_event_into_cube()
    popcache.update_feature()
    total_popularity = popcache.update_hypercube_estimate_value()
    popcache.update_cache_set()
    # popcache.print_cubes()
    # popcache.enumerate_hypercubes()
    #popcache.update_one_day()
    popcache.event_lastday_list = []
    popcache.cur_day = curDay
    oneDayEvents = []
    
    sumReward[0] += float(ue.Rh)
    sumReward[1] += float(ue.Rl)
    sumReward[2] += float(ue.Ro)
    if index % 10000 == 0 :
        psi = 0
        p = torch.from_numpy(env.p)
        for u in UEs:
            psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
        print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
        print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
        print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
        print()
        sumHitrate[int(index // 10000)]   = round((edgeHit+UEHit.sum())/(index+1),5)
        UEHitrate [int(index // 10000)]   = round(UEHit.sum()/(index+1),5)
        edgeHitrate [int(index // 10000)] = round(edgeHit/(index+1),5)
        privacyReduction [int(index // 10000)] = round(float(psi)/len(UEs),5)
psi = 0
p = torch.from_numpy(env.p)
for u in UEs:
    psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
print()
print("----------------------------------------------------------------")
print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5),"privacy:",round(float(psi)/len(UEs),5))
print("----------------------------------------------------------------")
print()
sumHitrate [int(round(index / 10000,0))]  = round((edgeHit+UEHit.sum())/(index+1),5)
UEHitrate  [int(round(index / 10000,0))]  = round(UEHit.sum()/(index+1),5)
edgeHitrate[int(round(index / 10000,0))]  = round(edgeHit/(index+1),5)
privacyReduction [int(round(index / 10000,0))] = round(float(psi)/len(UEs),5)

--Time: Wed Sep 22 17:45:29 2021   Index: 0   Loss: 0.0 --
Reward: [0. 0. 0.] total reward: 0.0
UEHitrate: 0.0  edgeHitrate 0.0 sumHitrate 0.0  privacy: 4.02118

--Time: Wed Sep 22 17:46:36 2021   Index: 10000   Loss: 0.0 --
Reward: [2.10529e+00 1.67560e-01 1.24000e-03] total reward: 2.27409
UEHitrate: 0.0037  edgeHitrate 0.37356 sumHitrate 0.37726  privacy: 1.89026

--Time: Wed Sep 22 17:47:43 2021   Index: 20000   Loss: 0.0 --
Reward: [1.58613 0.18593 0.00168] total reward: 1.77375
UEHitrate: 0.00415  edgeHitrate 0.41443 sumHitrate 0.41858  privacy: 1.86039

--Time: Wed Sep 22 17:48:49 2021   Index: 30000   Loss: 0.0 --
Reward: [1.33087 0.17906 0.00193] total reward: 1.51187
UEHitrate: 0.005  edgeHitrate 0.39949 sumHitrate 0.40449  privacy: 1.89512

--Time: Wed Sep 22 17:49:56 2021   Index: 40000   Loss: 0.0 --
Reward: [1.14319 0.1735  0.00197] total reward: 1.31866
UEHitrate: 0.00555  edgeHitrate 0.38692 sumHitrate 0.39247  privacy: 1.87911

--Time: Wed Sep 22 17:51:03 2021   Index:

In [6]:
hitrate1 = np.array([0.0107, 0.02, 0.0349, 0.0539 ,0.1065, 0.2135, 0.3443, 0.482,0.6706, 0.7719, 0.8585, 0.9425])
hitrate2 =np.array([0.0112, 0.0206, 0.0354, 0.0545, 0.1038, 0.2096, 0.3406, 0.4616,0.6197, 0.7354, 0.8397, 0.9395])
hitrate3 = np.array([0.0111, 0.0218, 0.0378, 0.056,0.1117, 0.2306, 0.3692, 0.4896,0.6159, 0.7307, 0.8357, 0.9388])
(UIT[(UIT[2]>=12)&(UIT[2]<18)].shape[0]*hitrate1+UIT[(UIT[2]>=18)&(UIT[2]<24)].shape[0]*hitrate2+UIT[(UIT[2]>=24)&(UIT[2]<30)].shape[0]*hitrate3)/UIT[(UIT[2]>=12)&(UIT[2]<30)].shape[0]

array([0.01091909, 0.02026291, 0.03511909, 0.05416291, 0.1053169 ,
       0.21179107, 0.34267871, 0.47306099, 0.64829629, 0.75590618,
       0.85026209, 0.94118544])

In [8]:
np.set_printoptions(linewidth=170)
hitrate1 = np.array([0.02222, 0.04083, 0.05233, 0.08623, 0.15234, 0.25517, 0.34928, 0.43627, 0.5698, 0.69672, 0.78608, 0.84388])
hitrate2 =np.array([0.02245, 0.03751, 0.04835, 0.06754, 0.10708, 0.19111, 0.25089, 0.31116, 0.43444, 0.57156, 0.64897, 0.72132])
hitrate3 = np.array([0.01563, 0.0258, 0.0496, 0.05668, 0.13091, 0.25474, 0.33844, 0.40537, 0.50362, 0.60923, 0.6901, 0.76312])
(UIT[(UIT[2]>=12)&(UIT[2]<18)].shape[0]*hitrate1+UIT[(UIT[2]>=18)&(UIT[2]<24)].shape[0]*hitrate2+UIT[(UIT[2]>=24)&(UIT[2]<30)].shape[0]*hitrate3)/UIT[(UIT[2]>=12)&(UIT[2]<30)].shape[0]

array([0.02232078, 0.03937522, 0.05058602, 0.07804029, 0.13250767, 0.22709976, 0.30616681, 0.38144846, 0.51048704, 0.64187655, 0.72600022, 0.79017583])